In [1]:
# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal import qgeometries
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from collections import OrderedDict

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Analysis
# from qiskit_metal.renderers.renderer_gds.gds_renderer import QGDSRenderer
# from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.quantization import EPRanalysis
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping
import pyEPR as epr
import numpy as np

# Set up chip dimensions 
design = design = designs.DesignPlanar()
design._chips['main']['size']['size_x'] = '4.6mm'
design._chips['main']['size']['size_y'] = '2.5mm'
design._chips['main']['size']['size_z'] = '-500um'
design._chips['main']['center']['center_y'] = '-1mm'
design._chips['main']['center']['center_x'] = '0mm'

# Resonator and feedline gap width (W) and center conductor width (S) from reference 2
design.variables['cpw_width'] = '10 um' #S from reference 2
design.variables['cpw_gap'] = '6 um' #W from reference 2

design.overwrite_enabled = True

# Open GUI
gui = MetalGUI(design)

###################
# Single feedline #
###################

# Driven Lauchpad 1
x1 = '-2mm'
y1 = '0mm'
launch_options1 = dict(chip='main', pos_x=x1, pos_y=y1, orientation='360', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP1 = LaunchpadWirebond(design, 'LP1', options = launch_options1)

# Driven Launchpad 2
x2 = '2mm'
y1 = '0mm'
launch_options2 = dict(chip='main', pos_x=x2, pos_y=y1, orientation='180', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP2 = LaunchpadWirebond(design, 'LP2', options = launch_options2)

# coupling resonator1 to feedline 
q_read1 = CoupledLineTee(design,'Q_Read_T1', options=dict(pos_x = '-1.3mm', pos_y = '0mm', 
                                                        orientation = '0',
                                                        coupling_space = '30um',                                                         
                                                        coupling_length = '300um',
                                                        open_termination = True))

# coupling resonator2 to feedline 
q_read2 = CoupledLineTee(design,'Q_Read_T2', options=dict(pos_x = '0mm', pos_y = '0mm', 
                                                        orientation = '0',
                                                        coupling_space = '30um',                                                         
                                                        coupling_length = '300um',
                                                        open_termination = True))

# coupling resonator3 to feedline 
q_read3 = CoupledLineTee(design,'Q_Read_T3', options=dict(pos_x = '1.3mm', pos_y = '0mm', 
                                                        orientation = '0',
                                                        coupling_space = '30um',                                                         
                                                        coupling_length = '300um',
                                                        open_termination = True))


gui.rebuild()
# Using path finder to connect the two launchpads
TL_LP1_T1 = RoutePathfinder(design, 'TL_LP1_T1', options = dict(chip='main', trace_width ='10um',
                                            trace_gap ='6um',
                                            fillet='99um',                                                            
                                            hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='LP1',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Q_Read_T1',
                                                    pin='prime_start')
                                            )))

TL_T_LP2 = RoutePathfinder(design, 'TL_T_LP2', options = dict(chip='main', trace_width ='10um',
        trace_gap ='6um',
                                            fillet='99um',
                                                                  
        hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Read_T1',
                                                    pin='prime_end'),
                                                end_pin=Dict(
                                                    component='Q_Read_T2',
                                                    pin='prime_start')
                                            )))

TL_T_LP3 = RoutePathfinder(design, 'TL_T_LP3', options = dict(chip='main', trace_width ='10um',
        trace_gap ='6um',
                                            fillet='99um',
                                                                  
        hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Read_T2',
                                                    pin='prime_end'),
                                                end_pin=Dict(
                                                    component='Q_Read_T3',
                                                    pin='prime_start')
                                            )))

TL_T_LP4 = RoutePathfinder(design, 'TL_T_LP4', options = dict(chip='main', trace_width ='10um',
        trace_gap ='6um',
                                            fillet='99um',
                                                                  
        hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Read_T3',
                                                    pin='prime_end'),
                                                end_pin=Dict(
                                                    component='LP2',
                                                    pin='tie')
                                            )))


# # Rebuild the GUI

######################
# lambda/4 resonator1#
######################

# First we define the two end-points
stg1 = ShortToGround(design, 'stg1', options=dict(chip='main', pos_x='-1.3mm',  pos_y='-1.7mm', orientation='-90'))

# Use RouteMeander to fix the total length of the resonator
rt_meander1 = RouteMeander(design, 'meander1',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='3.7mm',
        hfss_wire_bonds = True,
        fillet='99 um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
        start_pin=Dict(component='stg1', pin='open'),
        end_pin=Dict(component='Q_Read_T1', pin='second_end')), ))

######################
# lambda/4 resonator2#
######################
# First we define the two end-points
stg2 = ShortToGround(design, 'stg2', options=dict(chip='main', pos_x='0.0mm',  pos_y='-1.7mm', orientation='-90'))

# Use RouteMeander to fix the total length of the resonator
rt_meander2 = RouteMeander(design, 'meander2',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='4.2mm',
        hfss_wire_bonds = True,
        fillet='99 um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
        start_pin=Dict(component='stg2', pin='open'),
        end_pin=Dict(component='Q_Read_T2', pin='second_end')), ))

######################
# lambda/4 resonator3#
######################
stg3 = ShortToGround(design, 'stg3', options=dict(chip='main', pos_x='1.3mm',  pos_y='-1.7mm', orientation='-90'))

# Use RouteMeander to fix the total length of the resonator
rt_meander3 = RouteMeander(design, 'meander3',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='4.7mm',
        hfss_wire_bonds = True,
        fillet='99 um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
        start_pin=Dict(component='stg3', pin='open'),
        end_pin=Dict(component='Q_Read_T3', pin='second_end')), ))


# rebuild the GUI
gui.rebuild()

06:33PM 00s WARNING [__init__]: Pin open does not exist in component stg1. meander1 has not been built. Please check your pin_input values.
06:33PM 00s WARNING [__init__]: Pin open does not exist in component stg2. meander2 has not been built. Please check your pin_input values.
06:33PM 00s WARNING [__init__]: Pin open does not exist in component stg3. meander3 has not been built. Please check your pin_input values.


In [2]:
from PALACE_Simulation import PALACE_Simulation

sim = PALACE_Simulation('Capacitance', '3res', design, ports = [])
sim.run_simulation()

Creating geometry for Capacitance Simulation.
No ports for processing.
Geometry successfully built in Gmsh.
{'dielectric_gaps': 5, 'ground_plane': 6, 'far_field': 9, 'dielectric_substrate': 7, 'air_box': 8}
{1: 'metal_0', 2: 'metal_1', 3: 'metal_2', 4: 'metal_3'}
Finely meshing surfaces: [9, 10, 11, 12]
Mesh successfully built in Gmsh.


In [2]:
from GMSH_Geometrey_Builder import GMSH_Geometry_Builder

GGB = GMSH_Geometry_Builder(design)
metals, dielectric_gaps, design_objects = GGB._construct_geometry_in_GMSH()

1
2
3
4
5
6
7
8
9
[27 28 29 30 31 32 22 24 23 25  5  9  4  8  3  7  2  6  1]
[22 23 24 25 26  5  1  7  2  8  3  9  4  6]
<class 'list'>


ValueError: too many values to unpack (expected 3)

In [ ]:
#run the simulations
from SQDMetal.COMSOL.Model import COMSOL_Model
from SQDMetal.COMSOL.SimCapacitance import COMSOL_Simulation_CapMats
from SQDMetal.COMSOL.SimRFsParameter import COMSOL_Simulation_RFsParameters

#initialise the COMSOL engine
COMSOL_Model.init_engine()

#create COMSOL_Model object and give it a name
cmsl = COMSOL_Model('test')

simCapMats = COMSOL_Simulation_CapMats(cmsl)
sim_sParams = COMSOL_Simulation_RFsParameters(cmsl, adaptive = 'None')

cmsl.initialize_model(design, [simCapMats, sim_sParams], bottom_grounded = True, resolution = 10)

#Add metallic layers
cmsl.add_metallic(1, threshold=1e-10, fuse_threshold=1e-10)
cmsl.add_ground_plane(threshold=1e-10)
#cmsl.fuse_all_metals()

#assign ports
sim_sParams.create_port_CPW_on_Launcher('LP1')
sim_sParams.create_port_CPW_on_Launcher('LP2')

#build model
cmsl.build_geom_mater_elec_mesh(mesh_structure='Finer')

#plot model
cmsl.plot()

In [ ]:
#Draw shapely geomtries from qiskit-metal design to Gmsh
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import gmsh

#Get list of all components in the design
component_list = design.all_component_names_id()

for component in component_list:
    component_coords = component[0].qgeometry_table('poly')

""" element_components = LP1.qgeometry_table('poly')
coord_list1 = element_components.get_coordinates().to_numpy()
coord_list1.item((1,0))

coord_list2 = element_components.get_coordinates()
coord_list2.loc[1,'x']
 """




In [1]:
type(component_list)

NameError: name 'component_list' is not defined